# Scene Classification-Val
## 1. Preprocess-KerasFolderClasses
- Import pkg
- Extract zip file
- Preview "scene_classes.csv"
- Preview "scene_{0}_annotations_20170904.json"
- Test the image and pickle function
- Split data into serval pickle file

This part need jupyter notebook start with "jupyter notebook --NotebookApp.iopub_data_rate_limit=1000000000" (https://github.com/jupyter/notebook/issues/2287)

Reference:
- https://challenger.ai/competitions
- https://github.com/jupyter/notebook/issues/2287

### Import pkg

In [29]:
import numpy as np
import pandas as pd
# import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import seaborn as sns
%matplotlib inline
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix

In [30]:
from keras.utils.np_utils import to_categorical # convert to one-hot-encoding
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPool2D, BatchNormalization
from keras.optimizers import Adam
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import LearningRateScheduler, TensorBoard

In [31]:
# import zipfile
import os
import zipfile
import math
from time import time
from IPython.display import display
import pdb
import json
from PIL import Image
import glob
import pickle

### Extract zip file

In [32]:
input_path = 'input'
datasetName = 'validation'
date = '20170908'

datasetFolder = input_path + '\\data_{0}'.format(datasetName)
zip_path = input_path + '\\ai_challenger_scene_{0}_{1}.zip'.format(datasetName, date)
extract_path = input_path + '\\ai_challenger_scene_{0}_{1}'.format(datasetName, date)
image_path = extract_path + '\\scene_{0}_images_{1}'.format(datasetName, date)
scene_classes_path = extract_path + '\\scene_classes.csv'
scene_annotations_path = extract_path + '\\scene_{0}_annotations_{1}.json'.format(datasetName, date)

print(input_path)
print(datasetFolder)
print(zip_path)
print(extract_path)
print(image_path)
print(scene_classes_path)
print(scene_annotations_path)

input
input\data_validation
input\ai_challenger_scene_validation_20170908.zip
input\ai_challenger_scene_validation_20170908
input\ai_challenger_scene_validation_20170908\scene_validation_images_20170908
input\ai_challenger_scene_validation_20170908\scene_classes.csv
input\ai_challenger_scene_validation_20170908\scene_validation_annotations_20170908.json


In [33]:
if not os.path.isdir(extract_path):
    with zipfile.ZipFile(zip_path) as file:
        for name in file.namelist():
            file.extract(name, input_path)

### Preview "scene_classes.csv"

In [34]:
scene_classes = pd.read_csv(scene_classes_path, header=None)
display(scene_classes.head())

,0,1,2
0,0,航站楼,airport_terminal
1,1,停机坪,landing_field
2,2,机舱,airplane_cabin
3,3,游乐场,amusement_park
4,4,冰场,skating_rink


In [35]:
def get_scene_name(lable_number, scene_classes_path):
    scene_classes = pd.read_csv(scene_classes_path, header=None)
    return scene_classes.loc[lable_number, 2]
print(get_scene_name(0, scene_classes_path))

airport_terminal


### Preview "scene_{0}_annotations_20170904.json"

**This part need jupyter notebook start with "jupyter notebook --NotebookApp.iopub_data_rate_limit=1000000000"**
https://github.com/jupyter/notebook/issues/2287

In [36]:
with open(scene_annotations_path, 'r', encoding='utf-8') as file:
    content = ''
    for line in file:
        content = content + line
scene_annotations = json.loads(content)

In [37]:
#We get a list
print('scene_{0}_annotations.type: %{1}'.format(datasetName, type(scene_annotations)))
print('scene_{0}_annotations.shape: %{1}'.format(datasetName, len(scene_annotations)))
order = 0
print(scene_annotations[order])
print('label_id[{0}]:\t{1}'.format(order, scene_annotations[order]['label_id']))
print('image_id[{0}]:\t{1}'.format(order, scene_annotations[order]['image_id']))
print(type(scene_annotations[order]['label_id']))

scene_validation_annotations.type: %<class 'list'>
scene_validation_annotations.shape: %7120
{'image_id': '0c58107693263d32551209512d858246e925fe29.jpg', 'label_id': '18', 'image_url': 'http://m4.biz.itc.cn/pic/new/n/31/90/Img7919031_n.jpg'}
label_id[0]:	18
image_id[0]:	0c58107693263d32551209512d858246e925fe29.jpg
<class 'str'>


In [23]:
from  shutil import copy2

In [24]:
if not os.path.isdir(datasetFolder):
    os.mkdir(datasetFolder)
print(datasetFolder)

input\data_validation


In [25]:
%%time
length = len(scene_annotations)
# length = 2
cwd = os.getcwd()
print(cwd)
# Create class folder first, then copy files into folders
for i in range(80):
    class_dir = os.path.join(cwd, datasetFolder + '\\class-{0}'.format(str(i).zfill(2)))
    if not os.path.isdir(class_dir):
        os.mkdir(class_dir)
        
# print(os.listdir(os.path.join(cwd, datasetFolder)))
print(sorted(os.listdir(os.path.join(cwd, datasetFolder))))

E:\SceneClassification
['class-00', 'class-01', 'class-02', 'class-03', 'class-04', 'class-05', 'class-06', 'class-07', 'class-08', 'class-09', 'class-10', 'class-11', 'class-12', 'class-13', 'class-14', 'class-15', 'class-16', 'class-17', 'class-18', 'class-19', 'class-20', 'class-21', 'class-22', 'class-23', 'class-24', 'class-25', 'class-26', 'class-27', 'class-28', 'class-29', 'class-30', 'class-31', 'class-32', 'class-33', 'class-34', 'class-35', 'class-36', 'class-37', 'class-38', 'class-39', 'class-40', 'class-41', 'class-42', 'class-43', 'class-44', 'class-45', 'class-46', 'class-47', 'class-48', 'class-49', 'class-50', 'class-51', 'class-52', 'class-53', 'class-54', 'class-55', 'class-56', 'class-57', 'class-58', 'class-59', 'class-60', 'class-61', 'class-62', 'class-63', 'class-64', 'class-65', 'class-66', 'class-67', 'class-68', 'class-69', 'class-70', 'class-71', 'class-72', 'class-73', 'class-74', 'class-75', 'class-76', 'class-77', 'class-78', 'class-79']
Wall time: 1.08 

In [26]:
for i in range(length):
    fileName = image_path + '/' + scene_annotations[i]['image_id']
#     print(fileName)
    trainDir = os.path.join(cwd, datasetFolder + '\\class-{0}'.format(str(scene_annotations[i]['label_id']).zfill(2)))
#     print(trainDir)
    
    copy2(fileName, trainDir)

In [27]:
print('Done!')

Done!


In [28]:
# import os,sys
# cwd = os.getcwd()
# train_path = os.path.join(cwd, 'input\\train_data')
# folder_names=os.listdir(train_path)
# print(folder_names)
# for folder in folder_names:
#     subfolder = os.path.join(train_path, folder)
#     print(subfolder)
#     filenames = os.listdir(subfolder)
#     count = 1
#     for file in filenames:
#         old_name = subfolder + '\\' + file
#         new_name = subfolder + '\\' + str(count) + '.jpg'
#         print(old_name)
#         print(new_name)
        
#         count=count+1
#         os.rename(old_name, new_name)
